In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


In [0]:
cd A2

/content/gdrive/My Drive/A2


In [0]:
import numpy as np
image_x = 240
image_y = 320

In [0]:
def createBatches(listOfImages, labelsForThisFolder, batchSize):
  print("Begin making batches")
  listOfImageBatches = []
  listOfLabelBatches = []
  totalNumberOfImages = len(listOfImages)

  print("Total Number of Images in this lecture: ", totalNumberOfImages)
  
  pointer = 0
  while(pointer < totalNumberOfImages):
    if(pointer + batchSize <= totalNumberOfImages):
      listOfImageBatches.append(np.asarray(listOfImages[pointer:pointer+batchSize]))
      listOfLabelBatches.append(labelsForThisFolder[pointer:pointer+batchSize])
      
    else:
      if((totalNumberOfImages - batchSize) % batchSize > batchSize/3):
        holder_images = np.zeros((batchSize, image_x, image_y))  # this is the last batch where the number of images is less than the batch size; do padding
        holder_images[0:totalNumberOfImages - pointer, :, :] = np.asarray(np.asarray(listOfImages[pointer:totalNumberOfImages]))
        listOfImageBatches.append(holder_images)
      
        holder_labels = np.zeros((batchSize, 1))
        holder_labels[0:totalNumberOfImages - pointer, :] = labelsForThisFolder[pointer:totalNumberOfImages]
        listOfLabelBatches.append(holder_labels)

  return listOfImageBatches, listOfLabelBatches

In [0]:
import os
import cv2
import pandas as pd

batchSize = 500
listOfImageBatchesForAllLectures = []
listOfLabelBatchesForAllLectures = []

for lectureNumber in os.listdir("/content/gdrive/My Drive/A2/frames"):
  if('lec' in lectureNumber):
    listOfImages = []
    print("lectureNumber: ", lectureNumber)

    labelsForThisFolder = pd.read_csv("/content/gdrive/My Drive/A2/labels/{}.csv".format(lectureNumber), header = None)
    labelsForThisFolder = np.asarray(labelsForThisFolder)
    print("Number of slides in this folder: ", np.shape(labelsForThisFolder))
    if(np.size(labelsForThisFolder) > 700):
      print("Rejected")
      continue

    for imageName in os.listdir(os.path.join("/content/gdrive/My Drive/A2/frames", lectureNumber)):
      imageFile = cv2.imread(os.path.join("/content/gdrive/My Drive/A2/frames", lectureNumber, imageName))
      if(imageFile is None):
#         exists = os.path.isfile(os.path.join("/content/gdrive/My Drive/A2/frames", lectureNumber, imageName))
        listOfImages.append(np.zeros((240, 320)))
        continue

      image_gray = cv2.cvtColor(imageFile, cv2.COLOR_BGR2GRAY)
      image_gray_resized = cv2.resize(image_gray, (320, 240))
      listOfImages.append(np.asarray(image_gray_resized))

    print("All images and labels loaded.")

    listOfImageBatchesForThisLecture, listOfLabelBatchesForThisLecture = createBatches(listOfImages, labelsForThisFolder, batchSize)
    listOfImageBatchesForAllLectures += listOfImageBatchesForThisLecture
    listOfLabelBatchesForAllLectures += listOfLabelBatchesForThisLecture
    
    print("-----")

print("Total Number of batches in complete data: ", len(listOfImageBatchesForAllLectures))

lectureNumber:  lec22
Number of slides in this folder:  (411, 1)
All images and labels loaded.
Begin making batches
Total Number of Images in this lecture:  411


In [0]:
x = np.expand_dims(np.asarray(listOfImageBatchesForAllLectures), 4)
y = np.asarray(listOfLabelBatchesForAllLectures)
y = np.reshape(y, (-1, batchSize))

print(np.shape(x))
print(np.shape(y))

(1, 500, 240, 320, 1)
(1, 500)


In [0]:
for i in range(np.shape(y)[0]):
  for j in range(np.shape(y)[1]):
    if(y[i, j] == 1):
      y[i, j-2:j+3] = 1

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0, shuffle= True)

In [0]:
num_classes = 2

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, TimeDistributed, BatchNormalization, Bidirectional, CuDNNLSTM
from keras.layers.core import Dense, Activation, Dropout

model = Sequential()

model.add(TimeDistributed(Conv2D(32, (7, 7), strides = (5, 5), activation = 'relu', padding = 'same'), input_shape = (500, 240, 320, 1)))
model.add(TimeDistributed(Conv2D(32, (3, 3), kernel_initializer = "he_normal", activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides = (2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(Conv2D(64, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides = (2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(Conv2D(128, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides = (2, 2))))

model.add(TimeDistributed(Conv2D(256, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(Conv2D(256, (3, 3), padding = 'same', activation = 'relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides = (2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(Dropout(0.5))
model.add(Bidirectional(CuDNNLSTM(256, return_sequences = False)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(500, activation = 'softmax'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.optimizers import Adam

opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.99, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
model.fit(x_train, y_train, nb_epoch=15, batch_size=1, validation_split=0, verbose = 2)